In [35]:
import pandas as pd

# 5º ano C - Matemática
dados_mat_5C = {
    "Série": ["5º ANO"] * 6,
    "% Acertos": [21.4, 35.7, 35.7, 35.7, 39.3, 46.4],
    "Disciplina": ["Matemática - 2700"] * 6,
    "Conteúdo": [
        "Situações-problema do campo multiplicativo",
        "Multiplicação",
        "Divisão com números inteiros",
        "Situações-problema envolvendo os significados da divisão",
        "Equivalência entre unidades de medida de capacidade",
        "Divisão com números naturais"
    ],
    "Descritor": [
        "Resolver multiplicações em que o multiplicador é um número de até dois algarismos.",
        "Resolver problemas simples de contagem, envolvendo o princípio multiplicativo.",
        "Resolver problemas envolvendo os significados da divisão.",
        "Resolver problemas envolvendo os significados da divisão.",
        "Estabelecer equivalência entre litro e mililitro.",
        "Associar a divisão aos seus significados."
    ]
}

df_mat_5C = pd.DataFrame(dados_mat_5C)

# 5º ano D - Língua Portuguesa
dados_lp_5D = {
    "Série": ["5º ANO"] * 3,
    "% Acertos": [35.7, 46.4, 46.4],
    "Disciplina": ["Língua Portuguesa - 1100"] * 3,
    "Conteúdo": [
        "Tema de um texto",
        "Gênero textual: lenda",
        "Ortografia"
    ],
    "Descritor": [
        "Identificar o tema de um texto.",
        "Reconhecer diferentes gêneros textuais.",
        "Identificar padrões ortográficos na escrita das palavras."
    ]
}

df_lp_5D = pd.DataFrame(dados_lp_5D)

# 5º ano D - Matemática
dados_mat_5D = {
    "Série": ["5º ANO"] * 11,
    "% Acertos": [3.7, 29.6, 29.6, 33.3, 33.3, 33.3, 37.0, 37.0, 40.7, 40.7, 44.4],
    "Disciplina": ["Matemática - 2700"] * 11,
    "Conteúdo": [
        "Situações-problema do campo multiplicativo",
        "Equivalência entre unidades de medida de capacidade",
        "Multiplicação",
        "Divisão com números inteiros",
        "Situações-problema envolvendo os significados da divisão",
        "Subtração",
        "Divisão com números naturais",
        "Multiplicação por dois algarismos",
        "Equivalência entre unidades de medida de massa",
        "Figuras geométricas planas",
        "Unidades de medida de temperatura"
    ],
    "Descritor": [
        "Resolver multiplicações em que o multiplicador é um número de até dois algarismos.",
        "Estabelecer equivalência entre litro e mililitro.",
        "Resolver problemas simples de contagem, envolvendo o princípio multiplicativo.",
        "Resolver problemas envolvendo os significados da divisão.",
        "Resolver problemas envolvendo os significados da divisão.",
        "Resolver situação problema envolvendo adição e subtração de números inteiros.",
        "Associar a divisão aos seus significados.",
        "Resolver problemas envolvendo os significados da multiplicação.",
        "Estabelecer equivalência entre as unidades de medida: t/kg, kg/g e g/mg.",
        "Identificar figuras geométricas planas.",
        "Ler temperatura registrada em grau Celsius em termômetro digital."
    ]
}

df_mat_5D = pd.DataFrame(dados_mat_5D)

# Exibindo os DataFrames
print("5º ano C - Matemática")
print(df_mat_5C, "\n")
print("5º ano D - Língua Portuguesa")
print(df_lp_5D, "\n")
print("5º ano D - Matemática")
print(df_mat_5D)


5º ano C - Matemática
    Série  % Acertos         Disciplina  \
0  5º ANO       21.4  Matemática - 2700   
1  5º ANO       35.7  Matemática - 2700   
2  5º ANO       35.7  Matemática - 2700   
3  5º ANO       35.7  Matemática - 2700   
4  5º ANO       39.3  Matemática - 2700   
5  5º ANO       46.4  Matemática - 2700   

                                            Conteúdo  \
0         Situações-problema do campo multiplicativo   
1                                      Multiplicação   
2                       Divisão com números inteiros   
3  Situações-problema envolvendo os significados ...   
4  Equivalência entre unidades de medida de capac...   
5                       Divisão com números naturais   

                                           Descritor  
0  Resolver multiplicações em que o multiplicador...  
1  Resolver problemas simples de contagem, envolv...  
2  Resolver problemas envolvendo os significados ...  
3  Resolver problemas envolvendo os significados ...  
4  Estab

In [36]:
df_mat_5D

,Série,% Acertos,Disciplina,Conteúdo,Descritor
0,5º ANO,3.7,Matemática - 2700,Situações-problema do campo multiplicativo,Resolver multiplicações em que o multiplicador...
1,5º ANO,29.6,Matemática - 2700,Equivalência entre unidades de medida de capac...,Estabelecer equivalência entre litro e mililitro.
2,5º ANO,29.6,Matemática - 2700,Multiplicação,"Resolver problemas simples de contagem, envolv..."
3,5º ANO,33.3,Matemática - 2700,Divisão com números inteiros,Resolver problemas envolvendo os significados ...
4,5º ANO,33.3,Matemática - 2700,Situações-problema envolvendo os significados ...,Resolver problemas envolvendo os significados ...
5,5º ANO,33.3,Matemática - 2700,Subtração,Resolver situação problema envolvendo adição e...
6,5º ANO,37.0,Matemática - 2700,Divisão com números naturais,Associar a divisão aos seus significados.
7,5º ANO,37.0,Matemática - 2700,Multiplicação por dois algarismos,Resolver problemas envolvendo os significados ...
8,5º ANO,40.7,Matemática - 2700,Equivalência entre unidades de medida de massa,Estabelecer equivalência entre as unidades de ...
9,5º ANO,40.7,Matemática - 2700,Figuras geométricas planas,Identificar figuras geométricas planas.


Geração do Dashboard
(http://127.0.0.1:8060/)

In [37]:
df = pd.read_csv('/Users/lucas/repositories/Unicamp/ms942/Resultado por Conteúdo 2Bim - Matemática .csv')

In [38]:
df2 = pd.read_csv('/Users/lucas/repositories/Unicamp/ms942/Resultado por Conteúdo 2Bim - Lingua Portuguesa.csv')

In [44]:
df= pd.concat([df, df2], ignore_index=True)


In [45]:
df.head(10)

,Série,% Acertos,Disciplina,Conteúdo,Descritor,turma A
0,5º ANO A,28.0,Língua Portuguesa - 1100,Gênero textual: lenda,Reconhecer diferentes gêneros textuais.,NaN
1,5º ANO A,28.0,Língua Portuguesa - 1100,Verbos,"Identificar verbos, seus tempos e funções.",NaN
2,5º ANO A,40.0,Língua Portuguesa - 1100,Concordância nominal,Reconhecer o funcionamento de estruturas lingu...,NaN
3,5º ANO A,44.0,Língua Portuguesa - 1100,Tema de um texto,Identificar o tema de um texto.,NaN
4,5º ANO A,48.0,Língua Portuguesa - 1100,Características de gênero textual,Reconhecer os elementos constitutivos de um gê...,NaN
5,5º ANO A,48.0,Língua Portuguesa - 1100,Finalidade de um texto,Identificar a finalidade de textos de diferent...,NaN
6,5º ANO A,48.0,Língua Portuguesa - 1100,Ortografia,Identificar padrões ortográficos na escrita da...,NaN
7,5º ANO B,40.0,Língua Portuguesa - 1100,Tema de um texto,Identificar o tema de um texto.,NaN
8,5º ANO B,48.0,Língua Portuguesa - 1100,Gênero textual: lenda,Reconhecer diferentes gêneros textuais.,NaN
9,5º ANO B,48.0,Língua Portuguesa - 1100,Verbos,"Identificar verbos, seus tempos e funções.",NaN


In [47]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd


# ============================================================
# Leitura e unificação dos arquivos
# ============================================================
path_lp = '/Users/lucas/repositories/Unicamp/ms942/Resultado por Conteúdo 2Bim - Lingua Portuguesa.csv'
path_mat = "/Users/lucas/repositories/Unicamp/ms942/Resultado por Conteúdo 2Bim - Matemática .csv"

df_lp = pd.read_csv(path_lp)
df_mat = pd.read_csv(path_mat)

# Normaliza nomes de colunas
df_lp.columns = df_lp.columns.str.strip()
df_mat.columns = df_mat.columns.str.strip()

# Adiciona coluna Disciplina faltante (garantindo nomes consistentes)
df_lp["Disciplina"] = "Língua Portuguesa"
df_mat["Disciplina"] = "Matemática"

# Unifica tudo
df = pd.concat([df_lp, df_mat], ignore_index=True)

# Remove headers duplicados
df = df[df["% Acertos"].astype(str) != "% Acertos"]

# Padroniza nomes de colunas
df.columns = df.columns.str.replace("Conteúdo", "Conteudo", regex=False)
df.columns = df.columns.str.replace("Série", "Serie", regex=False)

# Limpa e converte coluna % Acertos
df["% Acertos"] = (
    df["% Acertos"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.replace("%", "", regex=False)
    .str.strip()
)

df["% Acertos"] = pd.to_numeric(df["% Acertos"], errors="coerce")


# ============================================================
# DASHBOARD
# ============================================================
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Desempenho por Conteúdo"),

    html.Div([
        html.Label("Série / Turma:"),
        dcc.Dropdown(
            id="dropdown-serie",
            options=[{"label": s, "value": s} for s in sorted(df["Serie"].unique())],
            value=sorted(df["Serie"].unique())[0],
            clearable=False
        ),
    ], style={"width": "30%", "display": "inline-block", "margin-right": "30px"}),

    html.Div([
        html.Label("Disciplina:"),
        dcc.Dropdown(
            id="dropdown-disciplina",
            options=[{"label": d, "value": d} for d in sorted(df["Disciplina"].unique())],
            value=sorted(df["Disciplina"].unique())[0],
            clearable=False
        ),
    ], style={"width": "30%", "display": "inline-block"}),

    dcc.Graph(id="grafico-barras")
])


# ============================================================
# CALLBACK
# ============================================================
@app.callback(
    Output("grafico-barras", "figure"),
    Input("dropdown-serie", "value"),
    Input("dropdown-disciplina", "value")
)
def atualizar_grafico(serie, disciplina):

    df_f = df[
        (df["Serie"] == serie) &
        (df["Disciplina"] == disciplina)
    ]

    fig = px.bar(
        df_f,
        x="Conteudo",
        y="% Acertos",
        color="Conteudo",
        text="% Acertos"
    )

    fig.update_layout(
        title=f"% de Acerto por Conteúdo — {disciplina} ({serie})",
        xaxis_title="Conteúdo",
        yaxis_title="% Acertos",
        legend_title="Conteúdos",
        yaxis=dict(range=[0, 100])
    )

    fig.update_traces(texttemplate="%{text:.1f}%", textposition="outside")

    return fig


# ============================================================
# Run
# ============================================================
if __name__ == "__main__":
    app.run(debug=True, port=8060)
